# 1. Information about the submission

## 1.1 Name and number of the assignment 

### Text categorization and argument mining task. HW2

## 1.2 Student name

### Nuzhnov Mark

## 1.3 Codalab user ID

### Nuzhnov_Mark

## 1.4 Additional comments

***Enter here** any additional comments which you would like to communicate to a TA who is going to grade this work not related to the content of your submission.*

# 2. Technical Report

*Use Section 2 to describe results of your experiments as you would do writing a paper about your results. DO NOT insert code in this part. Only insert plots and tables summarizing results as needed. Use formulas if needed do described your methodology. The code is provided in Section 3.*

## 2.1 Methodology 


BERT (Bidirectional Encoder Representations from Transformers) is a recent paper published by researchers at Google AI Language. In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix $Q$.   The keys and values are also packed together into matrices $K$ and $V$.  We compute the matrix of outputs as:                      
                                                                 
$$                                                                         
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V               
$$   

![alt text](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/qkv_explained-min.png)
Source: [Lena Voita's Lecture about Seq2Seq](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html)

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.                                            
$$    
\mathrm{MultiHead}(Q, K, V) = \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O    \\                                           
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)                                
$$                                                                                                                 

Where the projections are parameter matrices $W^Q_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in \mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in \mathbb{R}^{hd_v \times d_{\text{model}}}$.                                                                                                                                                                                             In this work I tried employ $h=3$, $h=5$, $h=8$ parallel attention layers, or heads. For each of these I use $d_k=d_v=d_{\text{model}}/h=64$. Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality. 

![alt text](https://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png)
Source: [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

## 2.2 Discussion of results

Discussing the results I would say that Baseline model (without multihead attention layer) gives a **0.3924 - F1_Stance_Detection** and **0.4517 - F1_Premise_Classification**. By adding MHA layer I was able to increased the score to **0.5713 - F1_Stance_Detection** and **0.6179 - F1_Premise_Classification**. Regarding to loss values we can see that in my case **val_argument_accuracy** was always higher than **val_stance_accuracy**, and loss was controversial. All graphs and losses can be found by the link to wandb below.

https://wandb.ai/smolenkovaea00/Text_categorization?workspace=user-smolenkovaea00

# 3. Code

*Enter here all code used to produce your results submitted to Codalab. Add some comments and subsections to navigate though your solution.*

*In this part you are expected to develop yourself a solution of the task and provide a reproducible code:*
- *Using Python 3;*
- *Contains code for installation of all dependencies;*
- *Contains code for downloading of all the datasets used*;
- *Contains the code for reproducing your results (in other words, if a tester downloads your notebook she should be able to run cell-by-cell the code and obtain your experimental results as described in the methodology section)*.


*As a result, you code will be graded according to these criteria:*
- ***Readability**: your code should be well-structured preferably with indicated parts of your approach (Preprocessing, Model training, Evaluation, etc.).*
- ***Reproducibility**: your code should be reproduced without any mistakes with “Run all” mode (obtaining experimental part).*


## 3.1 Requirements

In [ ]:
! pip install transformers
! pip install gdown
! pip install wandb

In [ ]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import MultiHeadAttention, Flatten, TimeDistributed
from tensorflow.keras.optimizers.experimental import AdamW
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.metrics import classification_report
import wandb
from wandb.keras import WandbCallback
#from google.colab import drive
#drive.mount('/content/drive')

seed_value = 42
os.environ['PYTHONHASHSEED']=str(seed_value)

In [ ]:
# Name of the BERT model to use
model_name = 'DeepPavlov/rubert-base-cased-sentence'

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config=config)

# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config=config, from_pt=True)

# Load the MainLayer
bert = transformer_model.layers[0]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## 3.2 Download the data

In [ ]:
!gdown 19LzgUlM3417TlG6bmo-eSY5vRAKw2ukR

Downloading...
From: https://drive.google.com/uc?id=19LzgUlM3417TlG6bmo-eSY5vRAKw2ukR
To: /kaggle/working/train_all.tsv
100%|███████████████████████████████████████| 1.54M/1.54M [00:00<00:00, 133MB/s]


## 3.3 Quarantine

### 3.3.1 Preprocessing

In [ ]:
CLASS_NAME = "quarantine"
# Import data from csv
whole_data = pd.read_csv('/kaggle/working/train_all.tsv', sep='\t')

# Train_test_split
test_size = 0.4
data, data_test = train_test_split(whole_data, test_size=test_size, random_state = seed_value)


#------------------------------------------------------------------------------------#
## Train
# Select required columns
data = data[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data[f'{CLASS_NAME}_stance'] = data['stance_label'].cat.codes
data[f'{CLASS_NAME}_argument'] = data['argument_label'].cat.codes

#------------------------------------------------------------------------------------#
## Test
# Select required columns
data_test = data_test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data_test['stance_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_stance'])
data_test['argument_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data_test[f'{CLASS_NAME}_stance'] = data_test['stance_label'].cat.codes
data_test[f'{CLASS_NAME}_argument'] = data_test['argument_label'].cat.codes

In [ ]:
#wandb.init(project="Text_categorization", name = "baseline_run", tags = ["Ruberta", "RB"])
# Ready output data for the model
test_y_stance = to_categorical(data_test[f'{CLASS_NAME}_stance'])
test_y_argument = to_categorical(data_test[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

### 3.3.2 Training Process

In [ ]:
# Build your model input
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"

__________________________________________________________________________________________________

 Layer (type)                   Output Shape         Param #     Connected to                     


 input_ids (InputLayer)         [(None, 256)]        0           []                               

                                                                                                  

 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['input_ids[0][0]']              

                                thPoolingAndCrossAt                                               

                                tentions(last_hidde                                               

                                n_state=(None, 256,                                               

                                 768),                                                            

                                 pooler_output=(Non           

In [ ]:
! wandb login --relogin Nikita4epuh
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Text_categorization", name = "baseline_run_quarantine", tags = ["Ruberta", "RB"])
epochs = 20
# Fit the model
history = model.fit(
    # x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    x={'input_ids': x['input_ids']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8]}, {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])
for epoch in range(epochs): 
    wandb.log({'loss': history.history['loss'][epoch],
               'argument_loss': history.history['argument_loss'][epoch],
               'stance_loss': history.history['stance_loss'][epoch],
               'argument_accuracy': history.history['argument_accuracy'][epoch],
               'stance_accuracy': history.history['stance_accuracy'][epoch],
               'val_loss': history.history['stance_accuracy'][epoch],
               'val_argument_loss': history.history['val_argument_loss'][epoch],
               'val_stance_loss': history.history['val_stance_loss'][epoch],
               'val_argument_accuracy': history.history['val_argument_accuracy'][epoch],
               'val_stance_accuracy': history.history['val_stance_accuracy'][epoch]}) 

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: smolenkovaea00. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20

672/672 [==============================] - ETA: 0s - loss: 1.7180 - argument_loss: 0.8148 - stance_loss: 0.9032 - argument_accuracy: 0.6868 - stance_accuracy: 0.6875


wandb: Adding directory to artifact (/content/wandb/run-20230418_200519-egdadxhc/files/model-best)... Done. 13.0s


672/672 [==============================] - 558s 674ms/step - loss: 1.7180 - argument_loss: 0.8148 - stance_loss: 0.9032 - argument_accuracy: 0.6868 - stance_accuracy: 0.6875 - val_loss: 1.5449 - val_argument_loss: 0.7069 - val_stance_loss: 0.8380 - val_argument_accuracy: 0.6250 - val_stance_accuracy: 0.6250

Epoch 2/20

672/672 [==============================] - ETA: 0s - loss: 1.4550 - argument_loss: 0.6857 - stance_loss: 0.7692 - argument_accuracy: 0.7499 - stance_accuracy: 0.7340


wandb: Adding directory to artifact (/content/wandb/run-20230418_200519-egdadxhc/files/model-best)... Done. 12.4s


672/672 [==============================] - 426s 633ms/step - loss: 1.4550 - argument_loss: 0.6857 - stance_loss: 0.7692 - argument_accuracy: 0.7499 - stance_accuracy: 0.7340 - val_loss: 0.3162 - val_argument_loss: 0.0847 - val_stance_loss: 0.2316 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000

Epoch 3/20

672/672 [==============================] - 358s 533ms/step - loss: 1.2980 - argument_loss: 0.6068 - stance_loss: 0.6912 - argument_accuracy: 0.7828 - stance_accuracy: 0.7621 - val_loss: 1.5579 - val_argument_loss: 0.7391 - val_stance_loss: 0.8189 - val_argument_accuracy: 0.6250 - val_stance_accuracy: 0.6250

Epoch 4/20

672/672 [==============================] - 354s 527ms/step - loss: 1.7145 - argument_loss: 0.8130 - stance_loss: 0.9014 - argument_accuracy: 0.6868 - stance_accuracy: 0.6873 - val_loss: 1.5648 - val_argument_loss: 0.7260 - val_stance_loss: 0.8388 - val_argument_accuracy: 0.6250 - val_stance_accuracy: 0.6250

Epoch 5/20

672/672 [=========================

### 3.3.3 Inference

In [ ]:
val_results = model.predict(x={'input_ids': test_x['input_ids']})

42/42 [==============================] - 23s 526ms/step


In [ ]:
data_test[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
data_test[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)

In [ ]:
print(classification_report(data_test[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support



           0       0.99      1.00      0.99       910

           1       0.00      0.00      0.00        33

           2       0.62      0.98      0.76       267

           3       0.00      0.00      0.00       134



    accuracy                           0.87      1344

   macro avg       0.40      0.49      0.44      1344

weighted avg       0.79      0.87      0.82      1344




In [ ]:
print(classification_report(data_test[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support



           0       0.99      1.00      0.99       910

           1       0.00      0.00      0.00        18

           2       0.85      0.97      0.91       375

           3       0.00      0.00      0.00        41



    accuracy                           0.94      1344

   macro avg       0.46      0.49      0.47      1344

weighted avg       0.91      0.94      0.92      1344




### 3.3.4 Prediction

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/HW_2/val_empty.tsv", sep='\t')
test.head()

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17041,> 26 марта его поместили на принудительный кар...,NaN,NaN,NaN,NaN,NaN,NaN
1,17057,И шевкунов вещает из телевизора про необходимо...,NaN,NaN,NaN,NaN,NaN,NaN
2,17058,Это результат его же лобировал до последнего ...,NaN,NaN,NaN,NaN,NaN,NaN
3,17071,При этом нормально обеспечены (к слову о якобы...,NaN,NaN,NaN,NaN,NaN,NaN
4,17079,для опасного врага нужен официальный карантин ...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

In [ ]:
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
test_results = model.predict(x={'input_ids': for_pred['input_ids']})

45/45 [==============================] - 29s 553ms/step


In [ ]:
test_results

{'stance': array([[-4.089829  , -0.33473855,  2.5395935 ,  1.3415891 ],
        [-4.0897512 , -0.3347472 ,  2.5395615 ,  1.3415637 ],
        [-4.0894866 , -0.33475885,  2.5394356 ,  1.3414531 ],
        ...,
        [ 5.198219  , -1.4057834 , -1.263636  , -1.1913619 ],
        [ 5.200686  , -1.4059892 , -1.2654437 , -1.1920764 ],
        [ 5.1925864 , -1.4060783 , -1.2590035 , -1.1900823 ]],
       dtype=float32),
 'argument': array([[-4.1424227 , -0.21485949,  2.9515023 ,  0.39686468],
        [-4.1423306 , -0.21488512,  2.9514856 ,  0.3968405 ],
        [-4.1420374 , -0.21497214,  2.9513922 ,  0.39675808],
        ...,
        [ 5.410088  , -1.8092142 , -0.41383114, -1.7783811 ],
        [ 5.4125404 , -1.8092424 , -0.41525313, -1.7785566 ],
        [ 5.40485   , -1.8096377 , -0.41075704, -1.7784284 ]],
       dtype=float32)}

In [ ]:
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)

<ipython-input-27-1f1aa18992cb>:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)

<ipython-input-27-1f1aa18992cb>:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)


In [ ]:
test_d[f'{CLASS_NAME}_stance'] -= 1
test_d[f'{CLASS_NAME}_argument'] -= 1

<ipython-input-28-c079fcfdc6d7>:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_stance'] -= 1

<ipython-input-28-c079fcfdc6d7>:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_argument'] -= 1


In [ ]:
test_d.head()

,text,quarantine_stance,quarantine_argument
0,> 26 марта его поместили на принудительный кар...,1,1
1,И шевкунов вещает из телевизора про необходимо...,1,1
2,Это результат его же лобировал до последнего ...,1,1
3,При этом нормально обеспечены (к слову о якобы...,1,1
4,для опасного врага нужен официальный карантин ...,1,1


In [ ]:
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"/content/drive/MyDrive/HW_2/val_predict_{CLASS_NAME}.tsv", sep='\t', index=None)

In [ ]:
CLASS_NAME = "quarantine"
df1 = pd.read_csv(f"/content/drive/MyDrive/HW_2/val_predict_{CLASS_NAME}.tsv", sep='\t')

### 3.3.5 new acrhitecture MHA

In [ ]:
# Build your model
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(256,), name='attention_mask', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]

# Add multi-head attention layer
attention_output = MultiHeadAttention(num_heads=4, key_dim=64)(bert_model, bert_model)

# Flatten the output from multi-head attention layer
flatten = Flatten()(attention_output)

# Apply dropout layer
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(flatten, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

#### 3.3.5.0 Training

In [ ]:
! wandb login --relogin Mark4epih
# Set an optimizer
optimizer = AdamW(
    learning_rate=5e-06,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Text_categorization", name = "Bert_attention_quarantine_ts_0.4_AdamW_5e6_heads_4_20epochs", tags = ["Ruberta_with_MHA", "RB"])
epochs = 3
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8], 'attention_mask': test_x['attention_mask'][:8]}, 
                     {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])

### 3.3.5.1 Inference

In [ ]:
val_results = model.predict(x={'input_ids': test_x['input_ids'], 'attention_mask': test_x['attention_mask']})
data_test[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
data_test[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)
print(classification_report(data_test[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0), classification_report(data_test[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

21/21 [==============================] - 14s 552ms/step

              precision    recall  f1-score   support



           0       1.00      0.98      0.99       468

           1       0.00      0.00      0.00        19

           2       0.59      0.99      0.74       126

           3       0.00      0.00      0.00        59



    accuracy                           0.87       672

   macro avg       0.40      0.49      0.43       672

weighted avg       0.80      0.87      0.83       672

               precision    recall  f1-score   support



           0       1.00      0.98      0.99       468

           1       0.00      0.00      0.00        15

           2       0.79      0.99      0.88       170

           3       0.00      0.00      0.00        19



    accuracy                           0.93       672

   macro avg       0.45      0.49      0.47       672

weighted avg       0.90      0.93      0.91       672




In [ ]:
val_results = model.predict(x={'input_ids': test_x['input_ids'], 'attention_mask': test_x['attention_mask']})
data_test[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
data_test[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)
print(classification_report(data_test[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0), classification_report(data_test[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

63/63 [==============================] - 38s 571ms/step

              precision    recall  f1-score   support



           0       0.97      0.99      0.98      1359

           1       0.00      0.00      0.00        47

           2       0.70      0.74      0.72       408

           3       0.41      0.41      0.41       202



    accuracy                           0.86      2016

   macro avg       0.52      0.53      0.53      2016

weighted avg       0.84      0.86      0.85      2016

               precision    recall  f1-score   support



           0       0.98      0.98      0.98      1359

           1       0.00      0.00      0.00        43

           2       0.82      0.95      0.88       543

           3       0.59      0.14      0.23        71



    accuracy                           0.93      2016

   macro avg       0.60      0.52      0.52      2016

weighted avg       0.90      0.93      0.91      2016




#### 3.3.5.2 Prediction

In [ ]:
test = pd.read_csv("/kaggle/working/val_empty.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})

In [ ]:
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_stance'] -= 1
test_d[f'{CLASS_NAME}_argument'] -= 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"/kaggle/working/val_predict_MHA_AdamW_lr_5e6_ts_0.4_num_4_20epochs{CLASS_NAME}.tsv", sep='\t', index=None)

In [ ]:
CLASS_NAME = "quarantine"
df1 = pd.read_csv(f"/content/drive/MyDrive/HW_2/val_predict_MHA_AdamW_lr_5e6_{CLASS_NAME}.tsv", sep='\t')

## 3.4 Masks

### 3.4.1 Preprocessing

In [ ]:
CLASS_NAME = "masks"
# Import data from csv
whole_data = pd.read_csv('/kaggle/working/train_all.tsv', sep='\t')

# Train_test_split
test_size = 0.4
data, data_test = train_test_split(whole_data, test_size=test_size, random_state = seed_value)


#------------------------------------------------------------------------------------#
## Train
# Select required columns
data = data[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data[f'{CLASS_NAME}_stance'] = data['stance_label'].cat.codes
data[f'{CLASS_NAME}_argument'] = data['argument_label'].cat.codes

#------------------------------------------------------------------------------------#
## Test
# Select required columns
data_test = data_test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data_test['stance_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_stance'])
data_test['argument_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data_test[f'{CLASS_NAME}_stance'] = data_test['stance_label'].cat.codes
data_test[f'{CLASS_NAME}_argument'] = data_test['argument_label'].cat.codes

In [ ]:
#wandb.init(project="Text_categorization", name = "baseline_run", tags = ["Ruberta", "RB"])
# Ready output data for the model
test_y_stance = to_categorical(data_test[f'{CLASS_NAME}_stance'])
test_y_argument = to_categorical(data_test[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
# Build your model input
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['input_ids[0][0]']              
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                               
                                 768),                                                            
                                 pooler_output=(Non                      

### 3.4.2 Training

In [ ]:
! wandb login --relogin VasyaBog
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Text_categorization", name = "baseline_run_masks", tags = ["Ruberta", "RB"])
epochs = 20
# Fit the model
history = model.fit(
    # x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    x={'input_ids': x['input_ids']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8]}, {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])
for epoch in range(epochs): 
    wandb.log({'loss': history.history['loss'][epoch],
               'argument_loss': history.history['argument_loss'][epoch],
               'stance_loss': history.history['stance_loss'][epoch],
               'argument_accuracy': history.history['argument_accuracy'][epoch],
               'stance_accuracy': history.history['stance_accuracy'][epoch],
               'val_loss': history.history['stance_accuracy'][epoch],
               'val_argument_loss': history.history['val_argument_loss'][epoch],
               'val_stance_loss': history.history['val_stance_loss'][epoch],
               'val_argument_accuracy': history.history['val_argument_accuracy'][epoch],
               'val_stance_accuracy': history.history['val_stance_accuracy'][epoch]}) 

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ERROR:wandb.sdk.internal.internal_api:500 response executing GraphQL.

ERROR:wandb.sdk.internal.internal_api:{"error":"driver: bad connection"}

wandb: Currently logged in as: smolenkovaea00. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20

672/672 [==============================] - ETA: 0s - loss: 1.7642 - argument_loss: 0.8188 - stance_loss: 0.9454 - argument_accuracy: 0.6588 - stance_accuracy: 0.6213


wandb: Adding directory to artifact (/content/wandb/run-20230419_070439-ofpqwwmf/files/model-best)... Done. 12.2s


672/672 [==============================] - 483s 633ms/step - loss: 1.7642 - argument_loss: 0.8188 - stance_loss: 0.9454 - argument_accuracy: 0.6588 - stance_accuracy: 0.6213 - val_loss: 1.7240 - val_argument_loss: 0.7653 - val_stance_loss: 0.9586 - val_argument_accuracy: 0.7500 - val_stance_accuracy: 0.6250

Epoch 2/20

672/672 [==============================] - 346s 514ms/step - loss: 1.1378 - argument_loss: 0.5051 - stance_loss: 0.6327 - argument_accuracy: 0.8558 - stance_accuracy: 0.7713 - val_loss: 1.9198 - val_argument_loss: 0.8926 - val_stance_loss: 1.0272 - val_argument_accuracy: 0.7500 - val_stance_accuracy: 0.6250

Epoch 3/20

672/672 [==============================] - ETA: 0s - loss: 1.1441 - argument_loss: 0.5083 - stance_loss: 0.6358 - argument_accuracy: 0.8558 - stance_accuracy: 0.7713


wandb: Adding directory to artifact (/content/wandb/run-20230419_070439-ofpqwwmf/files/model-best)... Done. 12.1s


672/672 [==============================] - 409s 609ms/step - loss: 1.1441 - argument_loss: 0.5083 - stance_loss: 0.6358 - argument_accuracy: 0.8558 - stance_accuracy: 0.7713 - val_loss: 1.6039 - val_argument_loss: 0.7321 - val_stance_loss: 0.8718 - val_argument_accuracy: 0.7500 - val_stance_accuracy: 0.6250

Epoch 4/20

672/672 [==============================] - 342s 509ms/step - loss: 1.1474 - argument_loss: 0.5106 - stance_loss: 0.6369 - argument_accuracy: 0.8559 - stance_accuracy: 0.7714 - val_loss: 1.6246 - val_argument_loss: 0.7632 - val_stance_loss: 0.8614 - val_argument_accuracy: 0.7500 - val_stance_accuracy: 0.6250

Epoch 5/20

672/672 [==============================] - 339s 505ms/step - loss: 1.1438 - argument_loss: 0.5080 - stance_loss: 0.6357 - argument_accuracy: 0.8554 - stance_accuracy: 0.7711 - val_loss: 1.7899 - val_argument_loss: 0.8472 - val_stance_loss: 0.9426 - val_argument_accuracy: 0.7500 - val_stance_accuracy: 0.6250

Epoch 6/20

252/672 [==========>..............

KeyboardInterrupt: ignored

### 3.4.3 Inference

In [ ]:
val_results = model.predict(x={'input_ids': test_x['input_ids']})

32/32 [==============================] - 20s 531ms/step


In [ ]:
data_test[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
data_test[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)
print(classification_report(data_test[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support



           0       0.99      0.99      0.99       534

           1       0.00      0.00      0.00        89

           2       0.60      1.00      0.75       287

           3       0.00      0.00      0.00        98



    accuracy                           0.81      1008

   macro avg       0.40      0.50      0.43      1008

weighted avg       0.70      0.81      0.74      1008




In [ ]:
print(classification_report(data_test[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support



           0       0.99      0.99      0.99       534

           1       0.00      0.00      0.00        56

           2       0.78      0.99      0.87       373

           3       0.00      0.00      0.00        45



    accuracy                           0.89      1008

   macro avg       0.44      0.49      0.47      1008

weighted avg       0.81      0.89      0.85      1008




In [ ]:
test = pd.read_csv("/content/drive/MyDrive/HW_2/val_empty.tsv", sep='\t')
test.head()

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17041,> 26 марта его поместили на принудительный кар...,NaN,NaN,NaN,NaN,NaN,NaN
1,17057,И шевкунов вещает из телевизора про необходимо...,NaN,NaN,NaN,NaN,NaN,NaN
2,17058,Это результат его же лобировал до последнего ...,NaN,NaN,NaN,NaN,NaN,NaN
3,17071,При этом нормально обеспечены (к слову о якобы...,NaN,NaN,NaN,NaN,NaN,NaN
4,17079,для опасного врага нужен официальный карантин ...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids']})

45/45 [==============================] - 25s 549ms/step


In [ ]:
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_stance'] -= 1
test_d[f'{CLASS_NAME}_argument'] -= 1

<ipython-input-13-d467d76760e1>:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)

<ipython-input-13-d467d76760e1>:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)

<ipython-input-13-d467d76760e1>:3: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



Se

In [ ]:
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"/content/drive/MyDrive/HW_2/val_predict_{CLASS_NAME}.tsv", sep='\t', index=None)
df2 = pd.read_csv(f"/content/drive/MyDrive/HW_2/val_predict_{CLASS_NAME}.tsv", sep='\t')

### 3.4.4 New Arch

In [ ]:
# Build your model
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(256,), name='attention_mask', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]

# Add multi-head attention layer
attention_output = MultiHeadAttention(num_heads=4, key_dim=64)(bert_model, bert_model)

# Flatten the output from multi-head attention layer
flatten = Flatten()(attention_output)

# Apply dropout layer
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(flatten, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 256,                      

### 3.4.5 Training

In [ ]:
! wandb login --relogin NikitaLoh
# Set an optimizer
optimizer = AdamW(
    learning_rate=5e-06,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Text_categorization", name = "Bert_attention_masks_20epochs_AdamW_lr_5e6_masks_num_4", tags = ["Ruberta_with_MHA", "RB"])
epochs = 20
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8], 'attention_mask': test_x['attention_mask'][:8]}, 
                     {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


argument_accuracy,▁██
argument_loss,█▁▁
epoch,▁▅█
loss,█▁▁
stance_accuracy,▁▇█
stance_loss,█▁▁
val_argument_accuracy,▁▁▁
val_argument_loss,█▂▁
val_loss,█▁▁
val_stance_accuracy,▁██
val_stance_loss,█▁▁


Epoch 1/20
672/672 [==============================] - ETA: 0s - loss: 0.9278 - argument_loss: 0.3746 - stance_loss: 0.5531 - argument_accuracy: 0.8783 - stance_accuracy: 0.7798

wandb: Adding directory to artifact (/kaggle/working/wandb/run-20230420_113722-b17zbcum/files/model-best)... Done. 13.9s


672/672 [==============================] - 332s 413ms/step - loss: 0.9278 - argument_loss: 0.3746 - stance_loss: 0.5531 - argument_accuracy: 0.8783 - stance_accuracy: 0.7798 - val_loss: 6.4571 - val_argument_loss: 3.3633 - val_stance_loss: 3.0938 - val_argument_accuracy: 0.3750 - val_stance_accuracy: 0.6250
Epoch 2/20
672/672 [==============================] - 213s 317ms/step - loss: 0.7414 - argument_loss: 0.2717 - stance_loss: 0.4697 - argument_accuracy: 0.9038 - stance_accuracy: 0.7945 - val_loss: 6.5955 - val_argument_loss: 3.4809 - val_stance_loss: 3.1146 - val_argument_accuracy: 0.2500 - val_stance_accuracy: 0.3750
Epoch 3/20
672/672 [==============================] - 209s 310ms/step - loss: 0.6589 - argument_loss: 0.2344 - stance_loss: 0.4245 - argument_accuracy: 0.9176 - stance_accuracy: 0.8282 - val_loss: 8.5854 - val_argument_loss: 4.3471 - val_stance_loss: 4.2382 - val_argument_accuracy: 0.2500 - val_stance_accuracy: 0.2500
Epoch 4/20
672/672 [==============================]

#### 3.4.5.1 Inference

In [ ]:
val_results = model.predict(x={'input_ids': test_x['input_ids'], 'attention_mask': test_x['attention_mask']})
data_test[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
data_test[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)
print(classification_report(data_test[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0), classification_report(data_test[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

32/32 [==============================] - 25s 653ms/step

              precision    recall  f1-score   support



           0       1.00      1.00      1.00       534

           1       0.41      0.21      0.28        89

           2       0.75      0.75      0.75       287

           3       0.28      0.41      0.33        98



    accuracy                           0.80      1008

   macro avg       0.61      0.59      0.59      1008

weighted avg       0.81      0.80      0.80      1008

               precision    recall  f1-score   support



           0       1.00      1.00      1.00       534

           1       0.33      0.54      0.41        56

           2       0.90      0.92      0.91       373

           3       0.00      0.00      0.00        45



    accuracy                           0.90      1008

   macro avg       0.55      0.61      0.58      1008

weighted avg       0.88      0.90      0.89      1008




#### 3.4.5.2 Predict

In [ ]:
CLASS_NAME = "masks"
test = pd.read_csv("/kaggle/working/val_empty.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_stance'] -= 1
test_d[f'{CLASS_NAME}_argument'] -= 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"/kaggle/working/val_predict_MHA_AdamW_lr_5e6_ts_0.4_num_4_20epochs_{CLASS_NAME}.tsv", sep='\t', index=None)

45/45 [==============================] - 15s 322ms/step


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

## 3.5 vaccines

### 3.5.1 Preprocessing

In [ ]:
CLASS_NAME = "vaccines"
# Import data from csv
whole_data = pd.read_csv('/kaggle/working/train_all.tsv', sep='\t')

# Train_test_split
test_size = 0.2
data, data_test = train_test_split(whole_data, test_size=test_size)


#------------------------------------------------------------------------------------#
## Train
# Select required columns
data = data[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data[f'{CLASS_NAME}_stance'] = data['stance_label'].cat.codes
data[f'{CLASS_NAME}_argument'] = data['argument_label'].cat.codes

#------------------------------------------------------------------------------------#
## Test
# Select required columns
data_test = data_test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data_test['stance_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_stance'])
data_test['argument_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data_test[f'{CLASS_NAME}_stance'] = data_test['stance_label'].cat.codes
data_test[f'{CLASS_NAME}_argument'] = data_test['argument_label'].cat.codes
#wandb.init(project="Text_categorization", name = "baseline_run", tags = ["Ruberta", "RB"])
# Ready output data for the model
test_y_stance = to_categorical(data_test[f'{CLASS_NAME}_stance'])
test_y_argument = to_categorical(data_test[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
# Build your model input
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['input_ids[0][0]']              
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                               
                                 768),                                                            
                                 pooler_output=(Non                      

### 3.5.2 Training

In [ ]:
! wandb login --relogin Markkrasav4ik
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Text_categorization", name = "baseline_run_vaccines", tags = ["Ruberta", "RB"])
epochs = 20
# Fit the model
history = model.fit(
    # x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    x={'input_ids': x['input_ids']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8]}, {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])
for epoch in range(epochs): 
    wandb.log({'loss': history.history['loss'][epoch],
               'argument_loss': history.history['argument_loss'][epoch],
               'stance_loss': history.history['stance_loss'][epoch],
               'argument_accuracy': history.history['argument_accuracy'][epoch],
               'stance_accuracy': history.history['stance_accuracy'][epoch],
               'val_loss': history.history['stance_accuracy'][epoch],
               'val_argument_loss': history.history['val_argument_loss'][epoch],
               'val_stance_loss': history.history['val_stance_loss'][epoch],
               'val_argument_accuracy': history.history['val_argument_accuracy'][epoch],
               'val_stance_accuracy': history.history['val_stance_accuracy'][epoch]}) 

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: smolenkovaea00. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20

672/672 [==============================] - ETA: 0s - loss: 0.8639 - argument_loss: 0.3964 - stance_loss: 0.4674 - argument_accuracy: 0.8729 - stance_accuracy: 0.8394


wandb: Adding directory to artifact (/content/wandb/run-20230419_080129-aqk0c2j9/files/model-best)... Done. 12.9s


672/672 [==============================] - 497s 639ms/step - loss: 0.8639 - argument_loss: 0.3964 - stance_loss: 0.4674 - argument_accuracy: 0.8729 - stance_accuracy: 0.8394 - val_loss: 0.0885 - val_argument_loss: 0.0322 - val_stance_loss: 0.0563 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000

Epoch 2/20

672/672 [==============================] - ETA: 0s - loss: 0.5012 - argument_loss: 0.2189 - stance_loss: 0.2823 - argument_accuracy: 0.9295 - stance_accuracy: 0.8718


wandb: Adding directory to artifact (/content/wandb/run-20230419_080129-aqk0c2j9/files/model-best)... Done. 12.9s


672/672 [==============================] - 411s 612ms/step - loss: 0.5012 - argument_loss: 0.2189 - stance_loss: 0.2823 - argument_accuracy: 0.9295 - stance_accuracy: 0.8718 - val_loss: 0.0812 - val_argument_loss: 0.0418 - val_stance_loss: 0.0394 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000

Epoch 3/20

672/672 [==============================] - 342s 509ms/step - loss: 0.5515 - argument_loss: 0.2473 - stance_loss: 0.3043 - argument_accuracy: 0.9213 - stance_accuracy: 0.8708 - val_loss: 0.1275 - val_argument_loss: 0.0490 - val_stance_loss: 0.0785 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000

Epoch 4/20

672/672 [==============================] - 339s 505ms/step - loss: 1.5166 - argument_loss: 0.7262 - stance_loss: 0.7904 - argument_accuracy: 0.7696 - stance_accuracy: 0.7664 - val_loss: 0.9052 - val_argument_loss: 0.4384 - val_stance_loss: 0.4669 - val_argument_accuracy: 0.8750 - val_stance_accuracy: 0.8750

Epoch 5/20

672/672 [=========================

KeyboardInterrupt: ignored

In [ ]:
! wandb login --relogin VasyaBog
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Text_categorization", name = "baseline_run_vaccines_2_epochs", tags = ["Ruberta", "RB"])
epochs = 2
# Fit the model
history = model.fit(
    # x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    x={'input_ids': x['input_ids']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8]}, {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])
for epoch in range(epochs): 
    wandb.log({'loss': history.history['loss'][epoch],
               'argument_loss': history.history['argument_loss'][epoch],
               'stance_loss': history.history['stance_loss'][epoch],
               'argument_accuracy': history.history['argument_accuracy'][epoch],
               'stance_accuracy': history.history['stance_accuracy'][epoch],
               'val_loss': history.history['stance_accuracy'][epoch],
               'val_argument_loss': history.history['val_argument_loss'][epoch],
               'val_stance_loss': history.history['val_stance_loss'][epoch],
               'val_argument_accuracy': history.history['val_argument_accuracy'][epoch],
               'val_stance_accuracy': history.history['val_stance_accuracy'][epoch]}) 

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: smolenkovaea00. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/2

672/672 [==============================] - ETA: 0s - loss: 0.5840 - argument_loss: 0.2581 - stance_loss: 0.3259 - argument_accuracy: 0.9200 - stance_accuracy: 0.8684


wandb: Adding directory to artifact (/content/wandb/run-20230419_092630-z1wiapq3/files/model-best)... Done. 12.6s


672/672 [==============================] - 489s 638ms/step - loss: 0.5840 - argument_loss: 0.2581 - stance_loss: 0.3259 - argument_accuracy: 0.9200 - stance_accuracy: 0.8684 - val_loss: 0.2219 - val_argument_loss: 0.0349 - val_stance_loss: 0.1870 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 0.8750

Epoch 2/2

672/672 [==============================] - 341s 508ms/step - loss: 0.5058 - argument_loss: 0.2165 - stance_loss: 0.2893 - argument_accuracy: 0.9334 - stance_accuracy: 0.8773 - val_loss: 0.2485 - val_argument_loss: 0.0328 - val_stance_loss: 0.2157 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 0.8750


### 3.5.3 Inference

In [ ]:
val_results = model.predict(x={'input_ids': test_x['input_ids']})
data_test[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
data_test[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)
print(classification_report(data_test[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0))

42/42 [==============================] - 24s 533ms/step

              precision    recall  f1-score   support



           0       1.00      1.00      1.00      1009

           1       0.00      0.00      0.00        89

           2       0.51      0.98      0.67       173

           3       0.00      0.00      0.00        73



    accuracy                           0.88      1344

   macro avg       0.38      0.50      0.42      1344

weighted avg       0.81      0.88      0.84      1344




In [ ]:
print(classification_report(data_test[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support



           0       1.00      1.00      1.00      1009

           1       0.00      0.00      0.00        59

           2       0.72      0.98      0.83       244

           3       0.00      0.00      0.00        32



    accuracy                           0.93      1344

   macro avg       0.43      0.50      0.46      1344

weighted avg       0.88      0.93      0.90      1344




In [ ]:
test = pd.read_csv("/content/drive/MyDrive/HW_2/val_empty.tsv", sep='\t')
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids']})

45/45 [==============================] - 28s 571ms/step


In [ ]:
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_stance'] -= 1
test_d[f'{CLASS_NAME}_argument'] -= 1

<ipython-input-11-d467d76760e1>:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)

<ipython-input-11-d467d76760e1>:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)

<ipython-input-11-d467d76760e1>:3: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



Se

In [ ]:
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"/content/drive/MyDrive/HW_2/val_predict_{CLASS_NAME}.tsv", sep='\t', index=None)

In [ ]:
CLASS_NAME = "vaccines"фы
df3 = pd.read_csv(f"/content/drive/MyDrive/HW_2/val_predict_{CLASS_NAME}.tsv", sep='\t')

### 3.5.4 New ARCH

In [ ]:
# Build your model
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(256,), name='attention_mask', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]

# Add multi-head attention layer
attention_output = MultiHeadAttention(num_heads=3, key_dim=64)(bert_model, bert_model)

# Flatten the output from multi-head attention layer
flatten = Flatten()(attention_output)

# Apply dropout layer
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(flatten, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 256,                      

#### 3.5.5 Training

In [ ]:
#! wandb login --relogin Nikita4epuh
# Set an optimizer
optimizer = AdamW(
    learning_rate=5e-06,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

#wandb.init(project="Text_categorization", name = "Bert_attention_vaccines_4epochs", tags = ["Ruberta_with_MHA", "RB"])
epochs = 20
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8], 'attention_mask': test_x['attention_mask'][:8]}, 
                     {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs) #callbacks=[WandbCallback()])

Epoch 1/20
672/672 [==============================] - 281s 337ms/step - loss: 1.6283 - argument_loss: 0.2790 - stance_loss: 1.3493 - argument_accuracy: 0.9140 - stance_accuracy: 0.8535 - val_loss: 1.4829 - val_argument_loss: 0.2677 - val_stance_loss: 1.2152 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 0.7500
Epoch 2/20
672/672 [==============================] - 210s 312ms/step - loss: 1.5175 - argument_loss: 0.1928 - stance_loss: 1.3246 - argument_accuracy: 0.9300 - stance_accuracy: 0.8703 - val_loss: 1.6284 - val_argument_loss: 0.3714 - val_stance_loss: 1.2570 - val_argument_accuracy: 0.8750 - val_stance_accuracy: 0.6250
Epoch 3/20
672/672 [==============================] - 208s 309ms/step - loss: 1.4702 - argument_loss: 0.1642 - stance_loss: 1.3060 - argument_accuracy: 0.9315 - stance_accuracy: 0.8656 - val_loss: 1.4259 - val_argument_loss: 0.1078 - val_stance_loss: 1.3182 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 0.6250
Epoch 4/20
672/672 [====================

In [ ]:
! wandb login --login Nikitathebest
wandb.init(project="Text_categorization", name = "Bert_attention_vaccines_20epochs_Adam_5e-6", tags = ["Ruberta_with_MHA", "RB"])

for epoch in range(epochs): 
    wandb.log({'loss': history.history['loss'][epoch],
               'argument_loss': history.history['argument_loss'][epoch],
               'stance_loss': history.history['stance_loss'][epoch],
               'argument_accuracy': history.history['argument_accuracy'][epoch],
               'stance_accuracy': history.history['stance_accuracy'][epoch],
               'val_loss': history.history['stance_accuracy'][epoch],
               'val_argument_loss': history.history['val_argument_loss'][epoch],
               'val_stance_loss': history.history['val_stance_loss'][epoch],
               'val_argument_accuracy': history.history['val_argument_accuracy'][epoch],
               'val_stance_accuracy': history.history['val_stance_accuracy'][epoch]}) 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Usage: wandb login [OPTIONS] [KEY]...
Try 'wandb login --help' for help.

Error: No such option: --login Did you mean --relogin?
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


#### 3.5.6 Inference

In [ ]:
val_results = model.predict(x={'input_ids': test_x['input_ids'], 'attention_mask': test_x['attention_mask']})
data_test[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
data_test[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)
print(classification_report(data_test[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0), classification_report(data_test[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

42/42 [==============================] - 28s 614ms/step

              precision    recall  f1-score   support



           0       0.75      1.00      0.86      1010

           1       0.00      0.00      0.00        78

           2       0.00      0.00      0.00       174

           3       0.00      0.00      0.00        82



    accuracy                           0.75      1344

   macro avg       0.19      0.25      0.21      1344

weighted avg       0.56      0.75      0.64      1344

               precision    recall  f1-score   support



           0       1.00      1.00      1.00      1010

           1       0.00      0.00      0.00        50

           2       0.74      1.00      0.85       248

           3       0.00      0.00      0.00        36



    accuracy                           0.93      1344

   macro avg       0.43      0.50      0.46      1344

weighted avg       0.89      0.93      0.91      1344




#### 3.5.7 Predict

In [ ]:
!gdown 1lUerv_gpQvo_e8Fl-flxEDtz77z_ZsLx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Downloading...
From: https://drive.google.com/uc?id=1lUerv_gpQvo_e8Fl-flxEDtz77z_ZsLx
To: /kaggle/working/val_empty.tsv
100%|█████████████████████████████████████████| 316k/316k [00:00<00:00, 102MB/s]


In [ ]:
CLASS_NAME = "vaccines"
test = pd.read_csv("/kaggle/working/val_empty.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1)
test_d[f'{CLASS_NAME}_stance'] -= 1
test_d[f'{CLASS_NAME}_argument'] -= 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"/kaggle/working/val_predict_MHA_Adam_5e_6_{CLASS_NAME}.tsv", sep='\t', index=None)

45/45 [==============================] - 18s 322ms/step


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

## 3.6 Concatinate all files with results for masks, vaccines, quarantine

In [ ]:
CLASS_NAME = "quarantine"
df1 = pd.read_csv(f"/content/drive/MyDrive/HW_2/val_predict_MHA_{CLASS_NAME}.tsv", sep='\t')
CLASS_NAME = "masks"
df2 = pd.read_csv(f"/content/drive/MyDrive/HW_2/val_predict_MHA_{CLASS_NAME}.tsv", sep='\t')
CLASS_NAME = "vaccines"
df3 = pd.read_csv(f"/content/drive/MyDrive/HW_2/val_predict_MHA_{CLASS_NAME}.tsv", sep='\t')


result = pd.merge(df1, df2, on="text")
result = pd.merge(result, df3, on="text")
result.to_csv("/content/drive/MyDrive/HW_2/val_predict_concat_MHA.tsv", sep='\t', index=None)

In [ ]:
!zip /content/drive/MyDrive/HW_2/val_predict_concat.zip /content/drive/MyDrive/HW_2/val_predict_concat.tsv

updating: content/drive/MyDrive/HW_2/val_predict_concat.tsv (deflated 73%)
